In [12]:
import random 
import json
import copy

f=open('parameters.json')
parametros=json.load(f)
f.close()
drivers=parametros["sides"]["drivers"]["names"]
aux=parametros["sides"]["teams"]["names"]
roles=['reserve', 'first', 'second']
res={}
for x in drivers:
     random.shuffle(aux)
     lista_aux=[]
     for y in aux:
             lista_aux.append({'driver_priority':roles[random.randint(0,2)],'team':y})
     res[x]=lista_aux
json_cont=json.dumps(res)
parametros["preferences"]["drivers"]=res
f=open('prefs_equipos_drivers.json','w+')
f.write(json_cont)
f.close()
res={}
for x in aux:
     dict_aux={}
     for y in roles:
             random.shuffle(drivers)
             dict_aux[y]=drivers
     res[x]=dict_aux
jsoni=json.dumps(res)
parametros["preferences"]["teams"]=res
f=open('prefesteams.json','w+')
f.write(jsoni)
f.close()
f=open('parameters.json',"w+")
f.write(json.dumps(parametros,indent=4))
f.close()

In [14]:
import json 
from collections import defaultdict
import sys 
import copy
def generador_preferencias(lista):
    i=0
    while i<len(lista):
        yield lista[i]
        i+=1

def get_index_of (driver,preferencias):
    result=sys.maxsize
    i=0
    while i<len(preferencias):
        if preferencias[i]==driver:
            result=i
            break
        i+=1
    return result


def get_index_driver (preferences,team,priority):
    result=sys.maxsize
    i=0
    while i<len(preferences):
        if preferences[i]["team"]==team and preferences[i]["driver_priority"]==priority :
            result=i
            break
        i+=1
    return result

def deferred_acceptance_drivers(drivers,teams):
    pending=list(drivers.keys())
    current_matchings={x:defaultdict(lambda:sys.maxsize) for x in teams.keys()}
    matchings={}
    unmatched=[]
    while len(pending)>0:
        desired_position=next(drivers[pending[0]],None)
        if (desired_position is not None):
            team=desired_position["team"]
            priority=desired_position["driver_priority"]
            index=get_index_of(pending[0],teams[team][priority])
            if current_matchings[team][priority]>index:
                if (current_matchings[team][priority]!=sys.maxsize):
                    pending.insert(1,teams[team][priority][current_matchings[team][priority]])
                current_matchings[team][priority]=index
                pending.pop(0)
        else:
             unmatched.append(pending.pop())
    for team,position in teams.items():
        dict_aux={}
        for x in position.keys():
            result="UNMATCHED"
            if (current_matchings[team][x]!=sys.maxsize):
                result=position[x][current_matchings[team][x]]
            dict_aux[x]=result
        matchings[team]=dict_aux
        ##matchings[team]={x:(position[x][current_matchings[team][x]] if current_matchings[team][x] >=0 else x:"UNMATCHED") for (x) in position.keys()}
    return matchings,unmatched

def deferred_acceptance_teams(teams,drivers):
    pending=[]
    aux={x:y.keys() for x,y in teams.items()}
    for x in aux.keys():
        for y in aux[x]:
            pending.append((x,y))
    for x,y in pending:
        teams[x][y]=generador_preferencias(teams[x][y])
    current_matchings=defaultdict(lambda:None)
    unmatched=[]
    while len(pending)>0:
        actual=pending[0]
        desired_position=next(teams[actual[0]][actual[1]],None)
        if (desired_position is not None):
            driver=desired_position
            index=get_index_driver(drivers[driver],actual[0],actual[1])
            if  not current_matchings[driver] or (current_matchings[driver] is not None and get_index_driver(drivers[driver],current_matchings[driver][0],current_matchings[driver][1])>index):
                if (current_matchings[driver]!=None):
                    pending.insert(1,current_matchings[driver])
                current_matchings[driver]=actual
                pending.pop(0)
        else:
            unmatched.append(pending.pop())
        ##matchings[team]={x:(position[x][current_matchings[team][x]] if current_matchings[team][x] >=0 else x:"UNMATCHED") for (x) in position.keys()}

    return current_matchings,unmatched

if __name__=="__main__":
    f=open("parameters.json")
    data=json.load(f)
    f.close()
    positions=data["sides"]["teams"]["roles"]
    matchings={x:[] for x in positions}
    teams_preferences= copy.deepcopy(data["preferences"]["teams"])
    drivers_preferences_drivers_side={x:generador_preferencias(y) for x,y in data["preferences"]["drivers"].items()}
    match_teams=deferred_acceptance_teams(teams_preferences,data["preferences"]["drivers"])
    teams_preferences= data["preferences"]["teams"]
    match_drivers=deferred_acceptance_drivers(drivers_preferences_drivers_side,teams_preferences)
    print(match_teams)
    print(match_drivers)

(defaultdict(<function deferred_acceptance_teams.<locals>.<lambda> at 0x0000016AC518B920>, {'Stoffel Vandoorne': ('racing bulls', 'first'), 'Mick Schumacher': ('alpine', 'reserve'), 'George Russel': ('alpine', 'first'), 'Lance Stroll': ('mercedes', 'first'), 'Sebastian Vettel': ('mclaren', 'first'), 'Oscar Piastri': ('hass', 'second'), 'Charles Leclerc': ('williams', 'first'), 'Daniel Ricciardo': ('ferrari', 'reserve'), 'Liam Lawson': ('ferrari', 'second'), 'Nicholas Latifi': ('mercedes', 'second'), 'Zhou Guanyu': ('aston martin', 'second'), 'Oliver Bearman': ('williams', 'reserve'), 'Robert Kubica': ('williams', 'second'), 'Max Verstappen': ('hass', 'first'), 'Paul Di Resta': ('kick sauber', 'reserve'), 'Carlos Sainz': ('red bull', 'second'), 'Fernando Alonso': ('hass', 'reserve'), 'Nyck de Vries': ('kick sauber', 'first'), 'Valteri Bottas': ('ferrari', 'first'), 'Isack Hadjar': ('mclaren', 'reserve'), 'SÃ©bastien Buemi': ('kick sauber', 'second'), 'Nico Hulkenberg': ('red bull', 'fir

In [32]:
import json
import copy
from DADriversMarket import deferred_acceptance_drivers, deferred_acceptance_teams, generador_preferencias

# Cargar archivo de parámetros
with open("parameters.json") as f:
    data = json.load(f)

# Preparar preferencias
teams_preferences = copy.deepcopy(data["preferences"]["teams"])
drivers_preferences = {
    driver: generador_preferencias(preflist)
    for driver, preflist in data["preferences"]["drivers"].items()
}

# Ejecutar algoritmos
drivers_proposing = deferred_acceptance_drivers(drivers_preferences, teams_preferences)
teams_preferences = data["preferences"]["teams"]  # restaurar porque se muta
teams_proposing = deferred_acceptance_teams(teams_preferences, data["preferences"]["drivers"])

# Funciones para imprimir
def imprimir_resultado_equipos(matchings_dict, unmatched):
    print("=== Matching Equipos Proponen ===")
    for team, role_dict in matchings_dict.items():
        for role, driver in role_dict.items():
            print(f"{team.title()} ({role.title()}): {driver}")
    print("\nUNMATCHED POSITIONS:")
    if unmatched:
        for item in unmatched:
            print(item)
    else:
        print("None")

def imprimir_resultado_pilotos(matchings_dict, unmatched):
    print("\n=== Matching Pilotos Proponen ===")
    for driver, assignment in matchings_dict.items():
        if assignment is None:
            print(f"{driver}: UNMATCHED")
        else:
            team, role = assignment
            print(f"{driver}: {team.title()} ({role.title()})")
    print("\nUNMATCHED DRIVERS:")
    if unmatched:
        for driver in unmatched:
            print(driver)
    else:
        print("None")

# Mostrar resultados
imprimir_resultado_equipos(*drivers_proposing)
imprimir_resultado_pilotos(*teams_proposing)


=== Matching Equipos Proponen ===
Racing Bulls (Reserve): Franco Colapinto
Racing Bulls (First): Stoffel Vandoorne
Racing Bulls (Second): Yuki Tsunoda
Ferrari (Reserve): Daniel Ricciardo
Ferrari (First): Valteri Bottas
Ferrari (Second): Liam Lawson
Aston Martin (Reserve): Lewis Hamilton
Aston Martin (First): Kevin Magnussen
Aston Martin (Second): Zhou Guanyu
Alpine (Reserve): Mick Schumacher
Alpine (First): George Russel
Alpine (Second): Kimi Antonelli
Mercedes (Reserve): Lando Norris
Mercedes (First): Lance Stroll
Mercedes (Second): Nicholas Latifi
Kick Sauber (Reserve): Paul Di Resta
Kick Sauber (First): Nyck de Vries
Kick Sauber (Second): SÃ©bastien Buemi
Hass (Reserve): Fernando Alonso
Hass (First): Max Verstappen
Hass (Second): Oscar Piastri
Red Bull (Reserve): Alex Albon
Red Bull (First): Nico Hulkenberg
Red Bull (Second): Carlos Sainz
Mclaren (Reserve): Isack Hadjar
Mclaren (First): Sebastian Vettel
Mclaren (Second): Esteban Ocon
Williams (Reserve): Oliver Bearman
Williams (Firs

In [44]:
import copy

# Asegurar que drivers_preferences sean listas
drivers_prefs = {
    driver: list(prefs)
    for driver, prefs in data["preferences"]["drivers"].items()
}

# Asegurar que teams_preferences sean listas
teams_prefs = {
    team: {
        role: list(pilots)
        for role, pilots in roles.items()
    }
    for team, roles in data["preferences"]["teams"].items()
}

# Función corregida para detectar blocking pairs
def hay_blocking_pairs(drivers_preferences, teams_preferences, matching_pilotos):
    for driver, current_assignment in matching_pilotos.items():
        prefs = drivers_preferences[driver]
        for option in prefs:
            team = option['team']
            role = option['driver_priority']
            if current_assignment == (team, role):
                break
            # Buscar quién ocupa ese puesto actualmente
            assigned_driver = None
            for d, assign in matching_pilotos.items():
                if assign == (team, role):
                    assigned_driver = d
                    break
            # Si el equipo no tiene preferencias válidas, salta
            if driver not in teams_preferences[team][role]:
                continue
            if assigned_driver is not None and assigned_driver not in teams_preferences[team][role]:
                continue
            # Calcular índice de preferencia
            index_proposed = teams_preferences[team][role].index(driver)
            index_current = (
                teams_preferences[team][role].index(assigned_driver)
                if assigned_driver is not None
                else float('inf')
            )
            if index_proposed < index_current:
                print(f"🔴 Blocking pair encontrado: {driver} y {team} ({role})")
                return True
    return False


# Verificación
if hay_blocking_pairs(drivers_prefs, teams_prefs, teams_proposing[0]):
    print("❌ Matching NO es estable.")
else:
    print("✅ Matching es estable. No hay blocking pairs.")


✅ Matching es estable. No hay blocking pairs.


In [46]:
for team, roles in teams_prefs.items():
    for role, prefs in roles.items():
        if "Pierre Gasly" in prefs:
            print(f"{team.title()} ({role.title()}): incluye a Pierre Gasly")


In [48]:
"Pierre Gasly" in data["preferences"]["drivers"]


True

In [50]:
data["preferences"]["drivers"].get("Pierre Gasly", [])


[{'driver_priority': 'second', 'team': 'racing bulls'},
 {'driver_priority': 'second', 'team': 'kick sauber'},
 {'driver_priority': 'first', 'team': 'red bull'},
 {'driver_priority': 'reserve', 'team': 'williams'},
 {'driver_priority': 'second', 'team': 'mclaren'},
 {'driver_priority': 'first', 'team': 'mercedes'},
 {'driver_priority': 'second', 'team': 'aston martin'},
 {'driver_priority': 'first', 'team': 'hass'},
 {'driver_priority': 'reserve', 'team': 'alpine'},
 {'driver_priority': 'second', 'team': 'ferrari'}]